# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

/home/mathyn/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
print((corpus))
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

[[242, 6, 26, 1, 63, 243], [11, 9, 584, 3, 67, 27, 244, 8, 585, 71, 14, 380, 21, 1], [381, 2, 8, 245, 112, 3, 49, 98, 57, 586, 4, 17, 587, 72, 1], [205, 14, 380, 9, 588, 19, 7, 17, 50, 246, 57, 589, 10], [7, 2, 44, 33, 1, 152, 8, 5, 205, 53, 11, 174, 246, 57], [30, 4, 9, 382, 10, 14, 247, 248, 15, 127, 15, 4, 58, 18, 1], [249, 250, 206, 14, 383, 27, 384, 2, 385, 386, 1, 590], [8, 387, 5, 591, 592, 59, 20, 388, 1, 389, 8, 138, 43, 2], [593, 1, 594, 46, 175, 5, 128, 63, 23, 595, 176, 207], [295, 71, 14], [55, 9, 112, 30, 27, 390, 10, 13, 596, 99, 11, 60, 7, 30], [27, 91, 31, 8, 1, 45, 3, 296, 1, 63, 83, 3, 297, 51, 64], [51, 64, 6, 92, 20, 298, 46, 4, 53, 7, 100, 597, 7], [598, 3, 14, 13, 4, 299, 3, 84, 599, 34, 28, 19, 34, 1, 66], [7, 22, 113, 78, 391, 19, 46, 1, 63, 600, 139, 5, 392], [31, 8, 114, 393, 251, 2, 129, 34, 7, 2, 79, 252, 21], [11, 394, 3, 14, 101, 18, 7, 601, 395, 14, 248, 13, 4, 17], [115, 130, 208, 5, 63, 23, 140, 5, 393, 251, 57, 5, 392], [3, 253, 31, 8, 7, 2, 602, 23, 

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:
# create inverted index to help recover the words from indexes
inverted_index = {}
for word, i in tokenizer.word_index.items():
    inverted_index[i] = word
    
print(inverted_index)

{1: 'the', 2: 'and', 3: 'to', 4: 'she', 5: 'a', 6: 'i', 7: 'it', 8: 'of', 9: 'was', 10: 'in', 11: 'alice', 12: 'you', 13: 'that', 14: 'her', 15: 'as', 16: 'said', 17: 'had', 18: 'for', 19: 'but', 20: 'be', 21: 'on', 22: 'all', 23: 'with', 24: 'little', 25: 'mouse', 26: 'down', 27: 'very', 28: 'this', 29: 'not', 30: 'so', 31: 'out', 32: 'if', 33: 'is', 34: 'at', 35: 't', 36: 's', 37: 'll', 38: 'how', 39: 'they', 40: 'about', 41: 'herself', 42: 'me', 43: 'up', 44: 'what', 45: 'way', 46: 'when', 47: 'like', 48: 'one', 49: 'do', 50: 'no', 51: 'oh', 52: 'went', 53: 'thought', 54: 'again', 55: 'there', 56: 'see', 57: 'or', 58: 'could', 59: 'would', 60: 'think', 61: 'them', 62: 'know', 63: 'rabbit', 64: 'dear', 65: 'were', 66: 'time', 67: 'get', 68: 'here', 69: 'must', 70: 'my', 71: 'by', 72: 'into', 73: 'found', 74: 'such', 75: 'began', 76: 'soon', 77: 'm', 78: 'quite', 79: 'then', 80: 'off', 81: 'now', 82: 'go', 83: 'say', 84: 'have', 85: 'which', 86: 'come', 87: 'dinah', 88: 'your', 89: 't

In [4]:
################################
# helper snippet
################################

m = np.matrix([[1, 2], [3, 4]])
print(m)
(rows, columns) = m.shape
for rowIdx in range(rows):
    s = m[rowIdx]
    total = s.sum()
    print(s.sum())
    print(np.divide(s, total))
    print(s.dtype)
    print(s.flatten())
    
    #for value in m[rowIdx, :]:
    #    print(value)
    #print(m[rowIdx, :])
    



[[1 2]
 [3 4]]
3
[[0.33333333 0.66666667]]
int64
[[1 2]]
7
[[0.42857143 0.57142857]]
int64
[[3 4]]


In [5]:
############################
# helper snippet 
############################
nan_indexes = [797, 1090, 1093, 1094, 1095, 1096, 1097]
for sequence in corpus:
    for nan_index in nan_indexes:
        if nan_index in sequence:
            print(sequence)
            break
    
print(tokenizer.word_index)

[797]
[1090]
[1093]
[1094]
[1095]
[1096]
[1097]
{'the': 1, 'and': 2, 'to': 3, 'she': 4, 'a': 5, 'i': 6, 'it': 7, 'of': 8, 'was': 9, 'in': 10, 'alice': 11, 'you': 12, 'that': 13, 'her': 14, 'as': 15, 'said': 16, 'had': 17, 'for': 18, 'but': 19, 'be': 20, 'on': 21, 'all': 22, 'with': 23, 'little': 24, 'mouse': 25, 'down': 26, 'very': 27, 'this': 28, 'not': 29, 'so': 30, 'out': 31, 'if': 32, 'is': 33, 'at': 34, 't': 35, 's': 36, 'll': 37, 'how': 38, 'they': 39, 'about': 40, 'herself': 41, 'me': 42, 'up': 43, 'what': 44, 'way': 45, 'when': 46, 'like': 47, 'one': 48, 'do': 49, 'no': 50, 'oh': 51, 'went': 52, 'thought': 53, 'again': 54, 'there': 55, 'see': 56, 'or': 57, 'could': 58, 'would': 59, 'think': 60, 'them': 61, 'know': 62, 'rabbit': 63, 'dear': 64, 'were': 65, 'time': 66, 'get': 67, 'here': 68, 'must': 69, 'my': 70, 'by': 71, 'into': 72, 'found': 73, 'such': 74, 'began': 75, 'soon': 76, 'm': 77, 'quite': 78, 'then': 79, 'off': 80, 'now': 81, 'go': 82, 'say': 83, 'have': 84, 'which':

In [6]:
#create co-occurrence matrix
#print(corpus[0])
coMatrix = np.zeros((V, V))
for sequence in corpus:
    for idx, termId in enumerate(sequence):
        sl = len(sequence)
        ## select left window
        leftw = sequence[max(idx - window_size_corpus, 0): idx]
        ## select right window
        rightw = sequence[idx + 1: min(idx + window_size_corpus + 1, sl)]
        # update co-occurrence matrix
        neighboors = leftw + rightw
        for neighbor in neighboors:
            coMatrix[termId, neighbor] += 1
        

# matrix normalization
(rows, columns) = coMatrix.shape
for rowIdx in range(rows):
    # ignore first row with 0 entries everywhere
    if (rowIdx > 0):
        row = coMatrix[rowIdx]
        total = row.sum()
        # avoid division by zero in words that have no neighboors
        if total > 0:
            coMatrix[rowIdx] = np.divide(row, total)


In [7]:
print(coMatrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02363823 0.0364851  ... 0.00051387 0.         0.        ]
 [0.         0.04752343 0.01204819 ... 0.00066934 0.00066934 0.00066934]
 ...
 [0.         0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.         0.         0.125      ... 0.         0.         0.        ]
 [0.         0.         0.25       ... 0.         0.         0.        ]]


In [8]:
#find cosine similarity to Alice, Dinah and Rabbit

#find the word vectors for Alice, Dinah, and Rabbit
aliceIdx = tokenizer.word_index['Alice'.lower()]
dinahIdx = tokenizer.word_index['Dinah'.lower()]
rabbitIdx = tokenizer.word_index['Rabbit'.lower()]

aliceVector = coMatrix[aliceIdx].reshape(1, -1)
dinahVector = coMatrix[dinahIdx].reshape(1, -1)
rabbitVector = coMatrix[rabbitIdx].reshape(1, -1)
print(aliceVector)
print(dinahVector)
print(rabbitVector)

cosAD = cosine_similarity(aliceVector, dinahVector)
print("Cosine similarity between Alice and Dinah: {}".format(cosAD))
cosAR = cosine_similarity(aliceVector, rabbitVector)
print("Cosine similarity between Alice and Rabbit: {}".format(cosAR))
cosDR = cosine_similarity(dinahVector, rabbitVector)
print("Cosine similarity between Dinah and Rabbit: {}".format(cosDR))


[[0.         0.02646503 0.03402647 ... 0.         0.         0.        ]]
[[0.         0.03448276 0.03448276 ... 0.         0.         0.        ]]
[[0.         0.13559322 0.00847458 ... 0.         0.         0.        ]]
Cosine similarity between Alice and Dinah: [[0.39360011]]
Cosine similarity between Alice and Rabbit: [[0.47890931]]
Cosine similarity between Dinah and Rabbit: [[0.29862324]]


In [9]:
#find the closest words to Alice
# create an array containing the cosine similarity values for alice and the rest
nbrs = nn(n_neighbors=6, algorithm='brute', metric='cosine').fit(coMatrix)
distances, indices = nbrs.kneighbors(aliceVector)
print(distances)
print(indices)


[[0.         0.18693993 0.22482059 0.25996004 0.26423806 0.2653115 ]]
[[11  4  7  1  5 41]]


In [10]:
# validate the computed distances and the indices

From the results we observe the six closest words to 'Alice'.

Word with index 11 corresponds to 'Alice' itself. 

The words for the remaining indexes are:
* 4: she
* 7: it
* 1: the
* 5: a
* 41: herself

Discussion of the drawbacks:

In [11]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

TypeError: list indices must be integers or slices, not tuple

In [12]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

ValueError: not enough values to unpack (expected 2, got 0)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [33]:
# prepare data for cbow

from typing import Tuple


def generate_data_cbow(corupus: list, window_size: int=2) -> Tuple[np.ndarray, np.ndarray]:
    """
    Create CBOW based on a corpus.
    Loop through the sentences and create bag of words based on the windows size.
    Args:
        corpus: list with words, split up by sentences in a nested list
        window_size: size of the bag (1 = one word after and one word before)
    """
    data = np.array([], dtype=np.int32)
    target = np.array([])
    
    for sentence in corpus:
        for x in range(0, len(sentence) - window_size*2):
            y = []
            append_data = sentence[x: x+(2*window_size)+1]
            append_data.pop(window_size)
            data = np.append(data, append_data)
            y.append(sentence[x+window_size])
            x = np.array([append_data])
            y = np_utils.to_categorical(y, V)
            yield(x, y)

data = generate_data_cbow(corpus, window_size)
print(list(data))

[(array([[242,   6,   1,  63]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[  6,  26,  63, 243]]), array([[0., 1., 0., ..., 0., 0., 0.]])), (array([[11,  9,  3, 67]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[  9, 584,  67,  27]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[584,   3,  27, 244]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[  3,  67, 244,   8]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[ 67,  27,   8, 585]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[ 27, 244, 585,  71]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[244,   8,  71,  14]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[  8, 585,  14, 380]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[585,  71, 380,  21]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[71, 14, 21,  1]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[381,   2, 245, 112]]), array([[0., 0., 0., ..., 0., 0., 0.]])), (array([[  2,   8, 112,   3]]), array([[0., 0., 0., ..., 0

In [34]:
#create CBOW model
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [35]:
#define loss function
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [ ]:
#train model
EPOCHS = 10

for idx in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_cbow(corpus, window_size):
        loss += cbow.train_on_batch(x, y)
    print("cbow window_size(2) losses: {} -> {}".format(idx, loss))

cbow window_size(2) losses: 0 -> 26363.16833114624
cbow window_size(2) losses: 1 -> 25933.366132736206
cbow window_size(2) losses: 2 -> 25667.70733422041
cbow window_size(2) losses: 3 -> 25467.461122959852
cbow window_size(2) losses: 4 -> 25315.893717318773
cbow window_size(2) losses: 5 -> 25184.273798137903
cbow window_size(2) losses: 6 -> 25062.217040851712
cbow window_size(2) losses: 7 -> 24949.215492635965
cbow window_size(2) losses: 8 -> 24843.922195628285


In [14]:
#prepare data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
            
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    # repeat the same word several times
                    in_words.append([word])
                    # add the context words
                    labels.append(words[i])
            if in_words != []:
                #print(in_words)
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
                #print(all_in)
                #print(all_in[0].shape)
                #print(all_out)
                #print(all_out[0].shape)
                #break
    return (all_in,all_out)

In [15]:
#save the preprocessed data of Skipgram
def save_skipgram_data(filename, x, y):
    f = open(filename ,'w')
    for input,outcome  in zip(x,y):
        input = np.concatenate(input)
        f.write(" ".join(map(str, list(input))))
        f.write(",")
        outcome = np.concatenate(outcome)
        f.write(" ".join(map(str,list(outcome))))
        f.write("\n")
    f.close()

In [16]:
#load the preprocessed Skipgram data
def generate_data_skipgram_from_file(filename):
    f = open(filename ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [17]:
SKIPGRAM_2WORDS = "data_skipgram_2words.txt"
x,y = generate_data_skipgram(corpus,2,V)
save_skipgram_data(SKIPGRAM_2WORDS, x, y)

SKIPGRAM_4WORDS = "data_skipgram_4words.txt"
x,y = generate_data_skipgram(corpus,4,V)
save_skipgram_data(SKIPGRAM_4WORDS, x, y)


In [18]:
#create Skipgram models
skipgram_2words_model = Sequential()
skipgram_2words_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_2words_model.add(Reshape((dim, )))
skipgram_2words_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_4words_model = Sequential()
skipgram_4words_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_4words_model.add(Reshape((dim, )))
skipgram_4words_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [19]:
#define loss function for Skipgrams
skipgram_2words_model.compile(loss='categorical_crossentropy', optimizer='adam')
#define loss function for Skipgram
skipgram_4words_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
#train Skipgram model
EPOCHS = 15

skipgram_2words_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_2WORDS):
        loss += skipgram_2words_model.train_on_batch(x, y)
    skipgram_2words_losses.append((ite, loss))
    print("skipgram 2words losses: {} -> {}".format(ite, loss))


skipgram_4words_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_4WORDS):
        loss += skipgram_4words_model.train_on_batch(x, y)
    skipgram_4words_losses.append((ite, loss))
    print("skipgram 4words losses: {} -> {}".format(ite, loss))


skipgram 2words losses: 0 -> 41058.83136129379
skipgram 2words losses: 1 -> 36788.37342321873
skipgram 2words losses: 2 -> 35254.39657962322
skipgram 2words losses: 3 -> 33901.695895552635
skipgram 2words losses: 4 -> 32700.642214894295
skipgram 2words losses: 5 -> 31647.69207084179
skipgram 2words losses: 6 -> 30742.98376274109
skipgram 2words losses: 7 -> 29980.631573796272
skipgram 2words losses: 8 -> 29348.7450029552
skipgram 2words losses: 9 -> 28832.518941432238
skipgram 2words losses: 10 -> 28415.487924717367
skipgram 2words losses: 11 -> 28083.13225750625
skipgram 2words losses: 12 -> 27819.53668270074
skipgram 2words losses: 13 -> 27615.19028233923
skipgram 2words losses: 14 -> 27453.668635244016
skipgram 4words losses: 0 -> 40965.09411597252
skipgram 4words losses: 1 -> 37543.71683096886
skipgram 4words losses: 2 -> 36535.442192196846
skipgram 4words losses: 3 -> 35616.158650279045
skipgram 4words losses: 4 -> 34753.0270178318
skipgram 4words losses: 5 -> 33963.15987730026
sk

In [21]:
def save_computed_vectors(filename, vectors):
    f = open(filename ,'w')
    f.write(" ".join([str(V-1),str(dim)]))
    f.write("\n")

    for word, i in tokenizer.word_index.items():
        f.write(word)
        f.write(" ")
        f.write(" ".join(map(str, list(vectors[i,:]))))
        f.write("\n")
    f.close()

In [22]:
#Save computed vectors
VECTORS_SKIPGRAM_2WORDS_FILENAME = 'vectors_skipgram_2words.txt'
skipgram_2words_model_vectors = skipgram_2words_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_2WORDS_FILENAME, skipgram_2words_model_vectors)

VECTORS_SKIPGRAM_4WORDS_FILENAME = 'vectors_skipgram_4words.txt'
skipgram_4words_model_vectors = skipgram_4words_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_4WORDS_FILENAME, skipgram_4words_model_vectors)

In [23]:
#create CBOW model with additional dense layer
cbow_dense = Sequential()
cbow_dense.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_dense.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_dense.add(Dense(input_dim=dim, units=dim, kernel_initializer='he_uniform', activation='relu'))
cbow_dense.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [24]:
#define loss function for CBOW + dense
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [25]:
#train model for CBOW + dense


In [26]:
#create Skipgram models with additional dense layer

skipgram_2words_modified_model = Sequential()
skipgram_2words_modified_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_2words_modified_model.add(Reshape((dim, )))
skipgram_2words_modified_model.add(Dense(input_dim=dim, units=dim, kernel_initializer='he_uniform', activation='relu'))
skipgram_2words_modified_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_4words_modified_model = Sequential()
skipgram_4words_modified_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_4words_modified_model.add(Reshape((dim, )))
skipgram_4words_modified_model.add(Dense(input_dim=dim, units=dim, kernel_initializer='he_uniform', activation='relu'))
skipgram_4words_modified_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))


In [27]:
#define loss function for Skipgram + dense
skipgram_2words_modified_model.compile(loss='categorical_crossentropy', optimizer='adam')
skipgram_4words_modified_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
#train model for Skipgram + dense
EPOCHS = 15

skipgram_2words_modified_model_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_2WORDS):
        loss += skipgram_2words_modified_model.train_on_batch(x, y)
    skipgram_2words_modified_model_losses.append((ite, loss))
    print("skipgram 2words modified losses: {} -> {}".format(ite, loss))


skipgram_4words_modified_model_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_4WORDS):
        loss += skipgram_4words_modified_model.train_on_batch(x, y)
    skipgram_4words_modified_model_losses.append((ite, loss))
    print("skipgram 4words modified losses: {} -> {}".format(ite, loss))


skipgram 2words modified losses: 0 -> 40310.90489411354
skipgram 2words modified losses: 1 -> 36838.86753368378
skipgram 2words modified losses: 2 -> 35278.3482542038
skipgram 2words modified losses: 3 -> 33884.94670379162
skipgram 2words modified losses: 4 -> 32731.41457927227
skipgram 2words modified losses: 5 -> 31925.219900131226
skipgram 2words modified losses: 6 -> 31412.38204920292
skipgram 2words modified losses: 7 -> 30910.54431039095
skipgram 2words modified losses: 8 -> 30460.545686781406
skipgram 2words modified losses: 9 -> 30086.18329155445
skipgram 2words modified losses: 10 -> 29835.962469637394
skipgram 2words modified losses: 11 -> 29442.911810219288
skipgram 2words modified losses: 12 -> 29272.471844144166
skipgram 2words modified losses: 13 -> 29070.991730734706
skipgram 2words modified losses: 14 -> 28942.192518793046
skipgram 4words modified losses: 0 -> 40184.313856601715
skipgram 4words modified losses: 1 -> 37079.07307600975
skipgram 4words modified losses: 2 -

In [29]:
#Save computed vectors
VECTORS_SKIPGRAM_2WORDS_MODIFIED_FILENAME = 'vectors_skipgram_2words_modified.txt'
skipgram_2words_modified_model_vectors = skipgram_2words_modified_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_2WORDS_MODIFIED_FILENAME, skipgram_2words_modified_model_vectors)

VECTORS_SKIPGRAM_4WORDS_MODIFIED_FILENAME = 'vectors_skipgram_4words_modified.txt'
skipgram_4words_modified_model_vectors = skipgram_4words_modified_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_4WORDS_MODIFIED_FILENAME, skipgram_4words_modified_model_vectors)

In [30]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'she': 4,
 'a': 5,
 'i': 6,
 'it': 7,
 'of': 8,
 'was': 9,
 'in': 10,
 'alice': 11,
 'you': 12,
 'that': 13,
 'her': 14,
 'as': 15,
 'said': 16,
 'had': 17,
 'for': 18,
 'but': 19,
 'be': 20,
 'on': 21,
 'all': 22,
 'with': 23,
 'little': 24,
 'mouse': 25,
 'down': 26,
 'very': 27,
 'this': 28,
 'not': 29,
 'so': 30,
 'out': 31,
 'if': 32,
 'is': 33,
 'at': 34,
 't': 35,
 's': 36,
 'll': 37,
 'how': 38,
 'they': 39,
 'about': 40,
 'herself': 41,
 'me': 42,
 'up': 43,
 'what': 44,
 'way': 45,
 'when': 46,
 'like': 47,
 'one': 48,
 'do': 49,
 'no': 50,
 'oh': 51,
 'went': 52,
 'thought': 53,
 'again': 54,
 'there': 55,
 'see': 56,
 'or': 57,
 'could': 58,
 'would': 59,
 'think': 60,
 'them': 61,
 'know': 62,
 'rabbit': 63,
 'dear': 64,
 'were': 65,
 'time': 66,
 'get': 67,
 'here': 68,
 'must': 69,
 'my': 70,
 'by': 71,
 'into': 72,
 'found': 73,
 'such': 74,
 'began': 75,
 'soon': 76,
 'm': 77,
 'quite': 78,
 'then': 79,
 'off': 80,
 'now': 81,
 'go': 82

In [31]:
## train nearest neighbors model
nn_skipgram_2words = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_2words_model_vectors)
nn_skipgram_4words = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_4words_model_vectors)
nn_skipgram_2words_modified = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_2words_modified_model_vectors)
nn_skipgram_4words_modified = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_4words_modified_model_vectors)


In [32]:
## --- SNIPPET USED TO UNDERSTAND SOME OPERATIONS ---- ##
print(skipgram_2words_modified_model_vectors.shape)
print(len(tokenizer.word_index))

(1183, 100)
1182


In [33]:
#Implement your own analogy function
# implementation of the function argmax(w.(w3 + w2 - w1)), with w - w3 =' w2 - w1
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.neighbors import NearestNeighbors as nn
#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)

def analogy(word1, word2, word3, word_vectors, nnmodel):
    vector_w1 = word_vectors[tokenizer.word_index[word1]]
    vector_w2 = word_vectors[tokenizer.word_index[word2]]
    vector_w3 = word_vectors[tokenizer.word_index[word3]]
    composed_vector = vector_w1 - vector_w2 + vector_w3
    distances, indices = nnmodel.kneighbors(composed_vector.reshape(1, -1))
    return (distances, indices)

Analyze the four different word embeddings

Implement your own function to perform the analogy task with.
    
Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    
Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.
    
Visualize your results and interpret your results


In [34]:
# I want to try with: melancholy - funny + smile = sad

(distances_skipgram_2words, indices_skipgram_2words) = analogy("cry", "funny", "smile", 
                                                               skipgram_2words_model_vectors, 
                                                              nn_skipgram_2words)

(distances_skipgram_4words, indices_skipgram_4words) = analogy("cry", "funny", "smile", 
                                                               skipgram_4words_model_vectors, 
                                                              nn_skipgram_4words)

In [35]:
#print(distances_skipgram_2words)
#print(indices_skipgram_2words)
for index in indices_skipgram_2words[0]:
    print(inverted_index[index])

NameError: name 'inverted_index' is not defined

In [ ]:
#print(distances_skipgram_4words)
#print(indices_skipgram_4words)
for index in indices_skipgram_4words[0]:
    print(inverted_index[index])

Comparison performance:

In [ ]:
#Visualization results trained word embeddings

TSNE(n_components=2).fit(np.array(skipgram_2words_model_vectors, skipgram_2words_modified_model_vectors))

Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings